<a href="https://colab.research.google.com/github/marcelodepaoli/21_ANNs/blob/main/00_No%C3%A7%C3%B5es_b%C3%A1sicas_de_sintaxe_do_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Noções básicas de sintaxe do Keras

No TensorFlow 2.0 o Keras é a principal API. Vamos trabalhar em um projeto de regressão simples para entender os fundamentos da sintaxe do Keras e adição de camadas.

## Os dados

Para aprender a sintaxe básica do Keras, usaremos um conjunto de dados falsos muito simples. Nas aulas subsequentes, focaremos em conjuntos de dados reais. Por enquanto, vamos nos concentrar na sintaxe do TensorFlow 2.0.

Vamos supor que esses dados sejam medições de algumas pedras raras, com 2 recursos de medição e um preço de venda. Nosso objetivo final seria tentar prever o preço de venda de uma nova pedra preciosa que acabamos de extrair do solo, a fim de tentar definir um preço justo no mercado.

### Carregando os Dados

In [ ]:
import pandas as pd

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir("drive/My Drive/Colab Notebooks/IA/21_ANNs")
os.listdir()

In [ ]:
df = pd.read_csv('DATA/fake_reg.csv')

In [ ]:
df.head()

### Explorando os dados

Vamos dar uma checada rápida nos dados, devemos ver uma forte correlação entre as características e o "preço" deste dataset inventado.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.pairplot(df)

Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Convertendo o Pandas Dataframe em Numpy Arrays para utilização no Keras

# Características (Features)
X = df[['feature1','feature2']].values

# Rótulo (Label)
y = df['price'].values

# Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

## Normalizando/dimensionando os dados

Nós normalizamos/dimensionamos os dados das características (features).

[Por que não precisamos dimensionar o rótulo (label)](https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
help(MinMaxScaler)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Aviso: para evitar vazamento de dados do conjunto de teste, apenas ajustamos nosso scaler ao conjunto de treinamento

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Sintaxe TensorFlow 2.0


## Opções de Importação

Existem várias maneiras de importar o Keras através do Tensorflow (essa é uma escolha de estilo extremamente pessoal, use qualquer método de importação de sua preferência). Usaremos o método apresentado na **documentação oficial do TF**.

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
help(Sequential)

## Criando um modelo

Existem duas maneiras de criar modelos por meio da API do TF 2 Keras: passar uma lista de camadas de uma só vez ou adicioná-las uma a uma.

Vamos mostrar os dois métodos (cabe a você escolher qual método prefere).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

### Modelo - como uma lista de camadas

In [ ]:
model = Sequential([
    Dense(units=2),
    Dense(units=2),
    Dense(units=2)
])

### Modelo - adicionando as camadas uma a uma

In [ ]:
model = Sequential()

model.add(Dense(2))
model.add(Dense(2))
model.add(Dense(2))

Vamos construir um modelo simples e, em seguida, compilá-lo definindo nosso solucionador (solver)

In [ ]:
model = Sequential()

model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(4,activation='relu'))

# Nó de saída final
model.add(Dense(1))

model.compile(optimizer='rmsprop',loss='mse')

### Escolhendo um otimizador e função de perda

Tenha em mente que tipo de problema você está tentando resolver:

    # Para um problema de classificação multiclasse
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Para um problema de classificação binária
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Para um problema de regressão de erro quadrado médio
    model.compile(optimizer='rmsprop',
                  loss='mse')

# Treinando

Abaixo estão algumas definições comuns que são necessárias conhecer (e entender) para utilizar corretamente o Keras:

* Amostra (Sample): um elemento de um conjunto de dados.
    * Exemplo: uma imagem é uma amostra em uma rede convolucional (convolutional network)
    * Exemplo: um arquivo de áudio é uma amostra para um modelo de reconhecimento de fala

* Lote (Batch): um conjunto de N amostras. As amostras em um lote são processadas de forma independente, em paralelo. Se estiver treinando, um lote resulta em apenas uma atualização para o modelo. Um lote geralmente aproxima a distribuição dos dados de entrada melhor do que uma única entrada. Quanto maior o lote, melhor a aproximação; no entanto, também é verdade que o lote levará mais tempo para ser processado e ainda resultará em apenas uma atualização. Para inferência (avaliar/prever), é recomendável escolher um tamanho de lote que seja o maior possível sem esgotar a memória (já que lotes maiores geralmente resultarão em avaliação/previsão mais rápida).

* Época (Epoch): um corte arbitrário (cutoff), geralmente definido como "uma passagem em todo o conjunto de dados", usado para separar o treinamento em fases distintas, o que é útil para registro e avaliação periódica.

* Ao usar **validation_data** ou **validation_split** com o método **fit** dos modelos Keras, a avaliação será executada no final de cada época (epoch).

* Dentro do Keras, existe a possibilidade de se adicionar callbacks especificamente projetados para serem executados no final de uma época. Exemplos disso são as mudanças na taxa de aprendizado (learning rate changes) e o ponto de verificação do modelo (model checkpointing - saving).

In [ ]:
model.fit(X_train,y_train,epochs=250)

## Avaliação

Vamos avaliar o desempenho do modelo em nosso conjunto de treinamento e nosso conjunto de teste. Podemos comparar esses dois desempenhos para verificar o overfitting.

In [ ]:
model.history.history

In [ ]:
loss = model.history.history['loss']

In [ ]:
sns.lineplot(x=range(len(loss)),y=loss)
plt.title("Training Loss per Epoch");

### Comparando a avaliação final (MSE) no conjunto de treinamento e no conjunto de teste.

In [ ]:
model.metrics_names

In [ ]:
training_score = model.evaluate(X_train,y_train,verbose=0)
test_score = model.evaluate(X_test,y_test,verbose=0)

In [ ]:
training_score

In [ ]:
test_score

### Avaliações Adicionais

In [ ]:
test_predictions = model.predict(X_test)

In [ ]:
test_predictions

In [ ]:
pred_df = pd.DataFrame(y_test,columns=['Test Y'])

In [ ]:
pred_df

In [ ]:
test_predictions = pd.Series(test_predictions.reshape(300,))

In [ ]:
test_predictions

In [ ]:
pred_df = pd.concat([pred_df,test_predictions],axis=1)

In [ ]:
pred_df.columns = ['Test Y','Model Predictions']

In [ ]:
pred_df

In [ ]:
sns.scatterplot(x='Test Y',y='Model Predictions',data=pred_df)

In [ ]:
pred_df['Error'] = pred_df['Test Y'] - pred_df['Model Predictions']

In [ ]:
sns.displot(pred_df['Error'],bins=50)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
mean_absolute_error(pred_df['Test Y'],pred_df['Model Predictions'])

In [ ]:
mean_squared_error(pred_df['Test Y'],pred_df['Model Predictions'])

In [ ]:
# Essencialmente a mesma coisa, diferença apenas devido à precisão
test_score

In [ ]:
#RMSE
test_score**0.5

# Prevendo em dados novos

E se víssemos uma nova pedra preciosa do chão? Qual deve ser o preço? Este procedimento é **exatamente** igual ao da previsão em novos dados de teste

In [ ]:
# [[Feature1, Feature2]]
new_gem = [[998,1000]]

In [ ]:
# Não esqueça de normalizar!
scaler.transform(new_gem)

In [ ]:
new_gem = scaler.transform(new_gem)

In [ ]:
model.predict(new_gem)

## Salvando e carregando um modelo

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('my_model.h5')  # cria um arquivo HDF5 com o nome 'my_model.h5'

In [ ]:
later_model = load_model('my_model.h5')

In [ ]:
later_model.predict(new_gem)